<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/Aus_Inclinometro_GKN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import os

def procesar_gkn_inclinometros(file_path):
    try:
        print(f"Procesando el archivo: {file_path}")
        with open(file_path, 'r') as file:
            lines = file.readlines()
            fecha = next((line.split(":")[1].strip() for line in lines if line.startswith("DATE")), None)
            nro_sonda = next((line.split(":")[1].strip() for line in lines if line.startswith("PROBE NO.")), None)
            hole_no = next((line.split(":")[1].strip() for line in lines if line.startswith("HOLE NO.")), None)
            start_idx = next((i + 2 for i, line in enumerate(lines) if '#READINGS:' in line), None)
            if start_idx is None:
                raise ValueError(f"No se encontraron lecturas en el archivo {file_path}")
            data = [line.strip().split(',') for line in lines[start_idx:]]
            df = pd.DataFrame(data, columns=['Profundidad', 'A+', 'A-', 'B+', 'B-'])
            df = df.apply(pd.to_numeric, errors='coerce')
            df['Fecha'] = pd.to_datetime(fecha, dayfirst=True, errors='coerce').strftime('%d/%m/%Y')
            df['Inclinómetro'] = hole_no
            df['Nro. Sonda'] = nro_sonda
            df = df[['Fecha', 'Inclinómetro', 'Profundidad', 'A+', 'A-', 'B+', 'B-', 'Nro. Sonda']]
            print(f"Archivo procesado correctamente: {file_path}")
            return df
    except Exception as e:
        print(f"Error procesando el archivo {file_path}: {e}")
        return None

def procesar_varios_gkn(file_list):
    dfs = []
    for file_path in file_list:
        df = procesar_gkn_inclinometros(file_path)
        if df is not None:
            dfs.append(df)
    if dfs:
        df_final = pd.concat(dfs, ignore_index=True)
        print("Todos los archivos fueron procesados correctamente.")
        return df_final
    else:
        print("No se pudieron procesar los archivos.")
        return None

In [12]:
import os
from google.colab import files

def seleccionar_fuente_gkn():
    print("Selecciona la fuente de tus archivos GKN:")
    print("1 - Subir ZIP con todos los archivos desde tu computadora")
    print("2 - Carpeta en Google Drive")
    print("3 - Carpeta en /content de Colab")
    print("4 - Seleccionar carpeta localmente (visual, solo en Colab)")
    fuente = input("Escribe 1, 2, 3 o 4 y presiona ENTER: ")
    return fuente.strip()

file_names = []
fuente = seleccionar_fuente_gkn()

if fuente == "1":
    print("Sube el archivo .zip que contiene todos los archivos GKN:")
    uploaded = files.upload()
    import zipfile
    for filename in uploaded.keys():
        if filename.endswith('.zip'):
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                zip_ref.extractall('GKN_folder')
            print("ZIP descomprimido en /content/GKN_folder")
        else:
            print("El archivo subido no es un .zip")
    carpeta_gkn = '/content/GKN_folder'
    file_names = [os.path.join(carpeta_gkn, f) for f in os.listdir(carpeta_gkn) if f.lower().endswith('.gkn')]
    print("Archivos GKN encontrados:", file_names)

elif fuente == "2":
    from google.colab import drive
    drive.mount('/content/drive')
    carpeta_gkn = input("Pega la ruta completa de la carpeta en Google Drive (ej: /content/drive/MyDrive/MisGKN): ").strip()
    if not os.path.exists(carpeta_gkn):
        raise ValueError("La carpeta especificada no existe.")
    file_names = [os.path.join(carpeta_gkn, f) for f in os.listdir(carpeta_gkn) if f.lower().endswith('.gkn')]
    print("Archivos GKN encontrados:", file_names)

elif fuente == "3":
    carpeta_gkn = input("Pega la ruta de la carpeta en /content (ej: /content/GKN): ").strip()
    if not os.path.exists(carpeta_gkn):
        raise ValueError("La carpeta especificada no existe.")
    file_names = [os.path.join(carpeta_gkn, f) for f in os.listdir(carpeta_gkn) if f.lower().endswith('.gkn')]
    print("Archivos GKN encontrados:", file_names)

elif fuente == "4":
    try:
        from google.colab import widgets
        import shutil
        print("Selecciona la carpeta con los archivos GKN en tu computadora.")
        # Usar el selector visual
        folder_uploaded = files.upload()
        local_folder = '/content/selected_folder'
        os.makedirs(local_folder, exist_ok=True)
        # Mueve los archivos subidos a una carpeta
        for filename in folder_uploaded.keys():
            shutil.move(filename, os.path.join(local_folder, filename))
        file_names = [os.path.join(local_folder, f) for f in os.listdir(local_folder) if f.lower().endswith('.gkn')]
        print("Archivos GKN encontrados:", file_names)
    except Exception as e:
        print("Funcionalidad visual de carpeta local no disponible. Usa ZIP o Drive para carpetas masivas.")
        file_names = []

else:
    print("Opción no válida. Ejecuta la celda de nuevo y elige 1, 2, 3 o 4.")

# Procesar todos los archivos encontrados (si hay)
if file_names:
    df_gkn = procesar_varios_gkn(file_names)
    if df_gkn is not None:
        display(df_gkn.head())
else:
    print("No se encontraron archivos para procesar.")

Selecciona la fuente de tus archivos GKN:
1 - Subir ZIP con todos los archivos desde tu computadora
2 - Carpeta en Google Drive
3 - Carpeta en /content de Colab
4 - Seleccionar carpeta localmente (visual, solo en Colab)
Escribe 1, 2, 3 o 4 y presiona ENTER: 4
Selecciona la carpeta con los archivos GKN en tu computadora.


Saving MD-IN1-150_001.gkn to MD-IN1-150_001 (2).gkn
Saving MD-IN1-150_002.gkn to MD-IN1-150_002 (2).gkn
Saving MD-IN1-150_003.gkn to MD-IN1-150_003 (2).gkn
Saving MD-IN1-150_004.gkn to MD-IN1-150_004 (2).gkn
Saving MD-IN1-150_005.gkn to MD-IN1-150_005 (2).gkn
Saving MD-IN1-150_006.gkn to MD-IN1-150_006 (2).gkn
Saving MD-IN1-150_007.gkn to MD-IN1-150_007 (2).gkn
Saving MD-IN1-150_008.gkn to MD-IN1-150_008 (2).gkn
Saving MD-IN1-150_009.gkn to MD-IN1-150_009 (2).gkn
Saving MD-IN1-150_010.gkn to MD-IN1-150_010 (2).gkn
Saving MD-IN1-150_011.gkn to MD-IN1-150_011 (2).gkn
Saving MD-IN1-150_012.gkn to MD-IN1-150_012 (2).gkn
Saving MD-IN1-150_012_Compass.GKS to MD-IN1-150_012_Compass (1).GKS
Saving MD-IN1-150_013.gkn to MD-IN1-150_013 (2).gkn
Saving MD-IN1-150_014.gkn to MD-IN1-150_014 (2).gkn
Saving MD-IN1-150_015.gkn to MD-IN1-150_015 (2).gkn
Saving MD-IN1-150_016.gkn to MD-IN1-150_016 (2).gkn
Saving MD-IN1-150_017.gkn to MD-IN1-150_017 (2).gkn
Saving MD-IN1-150_018.gkn to MD-IN1-150_018 (2).

,Fecha,Inclinómetro,Profundidad,A+,A-,B+,B-,Nro. Sonda
0,29/06/2022,MD-IN1-150,55.5,970,-994,-14,105,Probe-93
1,29/06/2022,MD-IN1-150,55.0,968,-983,-15,110,Probe-93
2,29/06/2022,MD-IN1-150,54.5,960,-976,-14,110,Probe-93
3,29/06/2022,MD-IN1-150,54.0,975,-992,-7,98,Probe-93
4,29/06/2022,MD-IN1-150,53.5,982,-996,5,90,Probe-93


In [15]:
from google.colab import files
import os

print("¿Dónde quieres guardar los archivos?")
print("1 - Descargar a mi computadora (PC)")
print("2 - Guardar en /content de Colab")
print("3 - Guardar en mi Google Drive")
opcion = input("Elige 1, 2 o 3: ").strip()

if 'df_gkn' in locals() and df_gkn is not None and not df_gkn.empty:
    nombres_inclinometro = df_gkn['Inclinómetro'].unique()
    print(f"Se encontraron los siguientes inclinómetros: {nombres_inclinometro}")
    rutas_guardadas = []
    for nombre in nombres_inclinometro:
        df_sel = df_gkn[df_gkn['Inclinómetro'] == nombre]
        base_nombre = f"inclinometro_{str(nombre).replace(' ','_').replace('/','-')}"
        archivo_xlsx = f"{base_nombre}.xlsx"
        archivo_csv = f"{base_nombre}.csv"

        if opcion == "1":
            # Guardar temporalmente en /content y descargar
            df_sel.to_excel(archivo_xlsx, index=False)
            df_sel.to_csv(archivo_csv, index=False)
            print(f"Descargando: {archivo_xlsx} y {archivo_csv}")
            files.download(archivo_xlsx)
            files.download(archivo_csv)
            rutas_guardadas.extend([archivo_xlsx, archivo_csv])

        elif opcion == "2":
            # Guardar solo en /content
            df_sel.to_excel(f"/content/{archivo_xlsx}", index=False)
            df_sel.to_csv(f"/content/{archivo_csv}", index=False)
            print(f"Guardado en /content: {archivo_xlsx} y {archivo_csv}")
            rutas_guardadas.extend([f"/content/{archivo_xlsx}", f"/content/{archivo_csv}"])

        elif opcion == "3":
            # Guardar en Google Drive (asegúrate de tener Drive montado)
            from google.colab import drive
            drive.mount('/content/drive', force_remount=True)
            ruta_drive = input("Escribe la ruta completa en tu Drive donde guardar los archivos (ej: /content/drive/MyDrive/MisGKN): ").strip()
            os.makedirs(ruta_drive, exist_ok=True)
            ruta_xlsx = os.path.join(ruta_drive, archivo_xlsx)
            ruta_csv = os.path.join(ruta_drive, archivo_csv)
            df_sel.to_excel(ruta_xlsx, index=False)
            df_sel.to_csv(ruta_csv, index=False)
            print(f"Guardado en Drive: {ruta_xlsx} y {ruta_csv}")
            rutas_guardadas.extend([ruta_xlsx, ruta_csv])

        else:
            print("Opción no válida.")
            break

    print("Archivos generados:", rutas_guardadas)
else:
    print("No hay datos procesados para guardar.")

¿Dónde quieres guardar los archivos?
1 - Descargar a mi computadora (PC)
2 - Guardar en /content de Colab
3 - Guardar en mi Google Drive
Elige 1, 2 o 3: 2
Se encontraron los siguientes inclinómetros: ['MD-IN1-150']
Guardado en /content: inclinometro_MD-IN1-150.xlsx y inclinometro_MD-IN1-150.csv
Archivos generados: ['/content/inclinometro_MD-IN1-150.xlsx', '/content/inclinometro_MD-IN1-150.csv']
